In [ ]:
# !pip install langchain-community
# !pip install psycopg2-binary
# !pip install ell-ai
# !pip install openai
!pip install anthropic

In [31]:
from langchain_community.llms import Ollama
from dataclasses import dataclass
import psycopg2

@dataclass
class Comment:
    author: str
    text: str
    comment_id: str 

# Fetch comments from database
def fetch_comments(conn, video_id):
    cur = conn.cursor()

    postgreSQL_select_Query = "SELECT author, text, id FROM youtube_comments WHERE video_id=%s limit 10"
    cur.execute(postgreSQL_select_Query, (video_id,))
    comment_records = cur.fetchall()

    cur.close()

    return [Comment(author=row[0], text=row[1], comment_id=row[2]) for row in comment_records]

In [32]:
import psycopg2
from psycopg2 import sql

# Define the function to insert sentiment data
def insert_sentiment_data(conn, comment_id, sentiment_score, sentiment_detected, llm_model):
    try:
        # Create a cursor object using the connection
        cursor = conn.cursor()

        # Define the SQL query for insertion
        insert_query = """
        INSERT INTO youtube_comment_sentiments (comment_id, sentiment_score, sentiment_detected, llm_model)
        VALUES (%s, %s, %s, %s)
        RETURNING id;
        """

        # Execute the insertion query with the provided data
        cursor.execute(insert_query, (comment_id, sentiment_score, sentiment_detected, llm_model))

        # Fetch the returned id of the inserted record (autoincrement ID)
        inserted_id = cursor.fetchone()[0]

        # Commit the transaction to make the changes in the database permanent
        conn.commit()

        # Close the cursor
        cursor.close()

        print(f"Sentiment data inserted with ID {inserted_id}")
        return inserted_id
    except Exception as e:
        # If something goes wrong, rollback and print the error
        conn.rollback()
        print(f"Error inserting sentiment data: {e}")
        return None

In [29]:
import ell
from openai import OpenAI


client = OpenAI(
    base_url='http://192.168.8.116:11434/v1/',
    api_key='ollama',
)
MODEL = 'mistral:7b-instruct-v0.3-fp16'
ell.config.verbose = False
ell.config.register_model(MODEL, client)

@ell.simple(model=MODEL, client=client, temperature=0.1)
def extract_comment_sentiment(video_summary: str, video_comment: str):
    """You are a helpful assistant that provides sentiment analysis on youtube comments. 
    When given a youtube video summary and comment, you provide the sentiment analysis.
    You respond w/ a JSON object containing 'sentiment_score'(-1 to 1) and 
    'sentiment_detected'(positive or negative). You respond only with JSON; you do not 
    offer an explanation for the sentiment analysis"""
    return f"Video Summary: {video_summary}\n\nVideo Comment: {video_comment}" 

In [33]:
import json 

video_id = "kjtbpWS-EjI"
video_summary = "The video involves a person preparing and cooking fish, including cleaning, seasoning, and cooking it, while providing commentary throughout the process. The person talks about enjoying the cooking experience, using various spices, handling the fish, and ensuring everything is seasoned well. There’s mention of serving the food to others, giving thanks for the meal, and noting that everyone enjoys it. The vibe of the video is casual, with some positivity and encouragement, including phrases like 'bless up' and 'good tastable.' It's clear that the person is passionate about what they’re doing and enjoys sharing the process with viewers."

conn = psycopg2.connect(
    database="social-data", 
    user='syran', 
    password='sk8erb01', 
    host='192.168.8.116')

comments = fetch_comments(conn, video_id)

# process comments
for comment in comments:
    comment_sentiment = extract_comment_sentiment(video_summary, comment.text)
    json_object = json.loads(comment_sentiment)
    inserted_id = insert_sentiment_data(conn, comment.comment_id, json_object['sentiment_score'], json_object['sentiment_detected'], MODEL)
    print(inserted_id)
    print(f"Comment: {comment.text}\nSentiment: {comment_sentiment}\n")
    break


Sentiment data inserted with ID 1
1
Comment: No Fowl “ME NUH KNOW” 🙄😒 Chro man 😂😂😂😂😂😂😂😂😂
Sentiment:  {"sentiment_score": 0.2, "sentiment_detected": "negative"}

